<a name="top"></a>
<br/>
# Machine learning thermodynamic perturbation theory (MLPT) - Distance
<br/>

**École Nationale Supérieure des Mines de Nancy**  
Project under the supervison of [Dario Rocca](http://crm2.univ-lorraine.fr/lab/fr/personnel/dario-rocca/) and [Fabien Pascale](https://www.researchgate.net/profile/Fabien_Pascale).  

Title: *High-accuracy materials modeling by machine learning quantum simulations*.  
Authors: [Lucas Lherbier](https://www.linkedin.com/in/lucas-lherbier/).

Last update: Feb 3rd, 2020.
<br/>

---
This notebook completes my project report. The Jupyter notebooks are divided in three parts : 
* the **[MLPT_position](MLPT_position.ipynb)** file: it creates the dataset whose features are the relative positions to an atom of reference.
* the **[MLPT_Distance](MLPT_Distance.ipynb)** file: it creates the dataset whose features are the distances of the neighbors from an atom of reference. For each configuration, all the atoms in the primitive cell are atoms of reference.
* the **[MLPT_Machine Learning](MLPT_MachineLearning.ipynb)** file: it applies machine learning algorithms to the datasets.

This notebook is the **MLPT_Distance** file.

---

### Imports

In [3]:
%matplotlib inline
import pandas as pd
import numpy as np

import timeit
import collections
from operator import itemgetter

import ase
from ase.io import read, write
from ase.neighborlist import NeighborList
from ase.neighborlist import natural_cutoffs

---
<a name="def"></a>

# Distance descriptors

## Files extraction

Our data deals with science materials. The data file includes one material, whose primitive cell is composed of 42 atoms. For each configuration, we get the coordinates of the atoms.
The molecule position changes with time, then the coordinates of the atoms change. 

In [4]:
atoms_2 = ase.io.read('MLPOS.19000',index=':2',format="xyz") #for test
atoms_200 = ase.io.read('MLPOS.19000',index='::95',format="xyz")
data_energy = pd.read_csv('CORRECTIONS', header = None, sep =' ', names = ['Id', 'Energy'] )  

## Neighbors

In [5]:
cutoffs = natural_cutoffs(atoms_2[1]) #Generate a radial cutoff for every atom based on covalent radii.
index = 41 #atom of reference for which we search the neighors
print('Lenght of the cutoffs:',len(cutoffs))

cutoffs= [0]*42
cutoffs[index]=4.0 #value of the radius
print('List of the cutoffs:',cutoffs)

nblist = NeighborList(cutoffs, self_interaction=False,bothways=True)
nblist.update(atoms_2[1])

indices, offsets = nblist.get_neighbors(index)
print("\nIndices: ",indices)

print("Search of the neighbors for the atom: ",index)
print("Symbol of the atom: ",atoms_2[1][index].symbol,offsets,atoms_2[1].positions[index])
print("\n")
for i, offset in zip(indices, offsets):
    print(i," ",atoms_2[1][i].symbol," ",
          atoms_2[1].get_distances(index,i),
          np.linalg.norm(atoms_2[1].positions[index] - (atoms_2[1].positions[i] + offset @ atoms_2[1].get_cell())),
          offset,atoms_2[1].positions[i] + offset @ atoms_2[1].get_cell())

Lenght of the cutoffs: 42
List of the cutoffs: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4.0]

Indices:  [19 12 14 35  0  7  9 17 18 21 22 23 24 25 26 27 29 30 31 37 38 39]
Search of the neighbors for the atom:  41
Symbol of the atom:  Si [[ 1  0  0]
 [ 1  0  0]
 [ 1  0  0]
 [ 1  0  0]
 [ 0  0  0]
 [ 0  0 -1]
 [ 1  0 -1]
 [ 0  0 -1]
 [ 0  0 -1]
 [ 0  0  0]
 [ 0  0 -1]
 [ 0  0 -1]
 [ 0  0 -1]
 [ 0  0  0]
 [ 0  0 -1]
 [ 0  0  0]
 [ 0  0  0]
 [ 0  0  0]
 [ 1  0 -1]
 [ 0  0 -1]
 [ 0  0 -1]
 [ 0  0 -1]] [4.86999972 1.95677207 6.49048597]


19   O   [5.54091632] 4.061347659519735 [1 0 0] [8.67581462 1.33608451 7.76526623]
12   O   [7.781965] 1.6398711411928233 [1 0 0] [6.10523442 1.52747042 7.47996699]
14   O   [5.85586085] 4.068104376914353 [1 0 0] [7.17147119 1.89102933 9.84434646]
35   Si   [6.30134554] 3.056229759038534 [1 0 0] [7.34620028 2.13336283 8.27312077]
0   Al   [3.22665845] 3.226658450990714 [0 0 

## Functions

We create several useful functions. In order to assess the quality of the functions, some prints are made and can be compared to the neighbors found at the [Neighbors](##Neighbors) section. The functions created are:
* `atom_get_neighbors`: return the list of the neighbors of an atom of reference, in a sphere of given radius, and for a specific configuration;  


* `nb_neighbors`: return the list of the neighbors sorted in ascending order and the number of neighbors for each type of atom; 


* `max_type_neighbors`: return the maximum numbers of the neighbors for each type of atom;


* `mol_get_neighbors`: return the list of the neighbors for all the atoms of reference for a specific configuration and also the maximums of neighbors for each atom;


* `all_config_neighbors`: return the list of the neighbors for each configuration, for all the atoms. Return also the maximum number of neighbors for each type of atom sorted alphabetically in a dictionary;


* `class_neighbors`: return the distances for each atom type in a dictionary, for a specific atom of reference;


* `get_list_distances`: return the list of the distances order by ascending for a specific atom of reference;


* `create_row`: return a row vector with all the distances for all the atoms of references for one configuration;


* `data_array`: return the final data frame;



In [6]:
def atom_get_neighbors(configuration, index, radius):
    """ Return the list of the neighbors of a specific configuration, configuration, from an atom of reference whose 
    index is index. The radius is radius, expressed in Ångström."""
    cutoffs=natural_cutoffs(configuration) #Generate a radial cutoff for every atom based on covalent radii.
    cutoffs= [0]*42
    cutoffs[index]= radius #choix du rayon

    nblist = NeighborList(cutoffs, self_interaction=False,bothways=True)
    nblist.update(configuration)
    indices, offsets = nblist.get_neighbors(index)
    
    list_neighbors = []
    for i, offset in zip(indices, offsets):
        list_neighbors.append([i,configuration[i].symbol,float(configuration.get_distances(index,i))])
    return list_neighbors
    
list_neighbors = atom_get_neighbors(atoms_2[1],41,4)
print('List of the neighbors for one spectifit atom of reference:\n', list_neighbors)

List of the neighbors for one spectifit atom of reference:
 [[19, 'O', 5.540916316997791], [12, 'O', 7.7819650009910974], [14, 'O', 5.855860846368702], [35, 'Si', 6.301345535078604], [0, 'Al', 3.226658450990714], [7, 'O', 7.2637727153269545], [9, 'O', 10.615814078724782], [17, 'O', 9.558257708994331], [18, 'O', 8.240781936512354], [21, 'O', 4.018323624679267], [22, 'O', 6.883358416437047], [23, 'O', 8.342630041336772], [24, 'O', 6.398725961359146], [25, 'O', 4.597726319507452], [26, 'O', 8.275467725041082], [27, 'O', 4.198413268295706], [29, 'O', 1.5787762843840065], [30, 'O', 1.6792019993926566], [31, 'Si', 10.370024419115662], [37, 'Si', 8.165082114215778], [38, 'Si', 6.077268148662403], [39, 'Si', 7.567778645771182]]


In [7]:
def nb_neighbors(list_neighbors):
    """ From the list of the neighbors, return the list of neighbors sorted in ascending order and the number of neighbors for each type of atom 
    in a dictionary."""
    list_neighbors = sorted(list_neighbors, key=itemgetter(2))
    list_types = []
    for i in list_neighbors : 
        list_types.append(i[1])
    list_types = dict(collections.Counter(list_types))
    return list_neighbors,list_types

list_neighbors,list_types = nb_neighbors(list_neighbors)
print('List of the neighbors sorted in ascending order:',list_neighbors)
print('Number of neighbors for each atom type:',list_types)

List of the neighbors sorted in ascending order: [[29, 'O', 1.5787762843840065], [30, 'O', 1.6792019993926566], [0, 'Al', 3.226658450990714], [21, 'O', 4.018323624679267], [27, 'O', 4.198413268295706], [25, 'O', 4.597726319507452], [19, 'O', 5.540916316997791], [14, 'O', 5.855860846368702], [38, 'Si', 6.077268148662403], [35, 'Si', 6.301345535078604], [24, 'O', 6.398725961359146], [22, 'O', 6.883358416437047], [7, 'O', 7.2637727153269545], [39, 'Si', 7.567778645771182], [12, 'O', 7.7819650009910974], [37, 'Si', 8.165082114215778], [18, 'O', 8.240781936512354], [26, 'O', 8.275467725041082], [23, 'O', 8.342630041336772], [17, 'O', 9.558257708994331], [31, 'Si', 10.370024419115662], [9, 'O', 10.615814078724782]]
Number of neighbors for each atom type: {'O': 16, 'Al': 1, 'Si': 5}


In [8]:
def max_type_neighbors(list_dic) : 
    """ From a list of a dictionary with the number of neighbors for each type of atom, return the maximums of these 
    numbers for each type of atom in a dictionary."""
    dic_neighbors_max = list_dic[0]
    for dic_neighbors_atom_i in list_dic : 
        for key, val in dic_neighbors_atom_i.items():
            if key in dic_neighbors_max : 
                if val > dic_neighbors_max[key] : 
                    dic_neighbors_max[key] = val
            else : 
                dic_neighbors_max[key] = val
    return dic_neighbors_max


def mol_get_neighbors(configuration , radius) : 
    """ From a specific configuration, configuration, return the list of the neighbors for all the atom in a sphere of
    radius radius. Return also the dictionary of the maximums of neighbors for each atom type."""
    list_all_neighbors = []
    list_all_types = []
    for center in range(42) : 
        list_neighbors = atom_get_neighbors(configuration, center, radius)
        list_neighbors,list_types = nb_neighbors(list_neighbors)
        list_all_neighbors.append(list_neighbors)
        list_all_types.append(list_types)
    list_all_types = max_type_neighbors(list_all_types)
    return list_all_neighbors,list_all_types

list_all_neighbors,list_all_types = mol_get_neighbors(atoms_2[1],4)
print('Size of list_all_neighbors = ', len(list_all_neighbors))
print('Size of the elements of list_all_neighbors = ', len(list_all_neighbors[0]))
print('List of all the neighbors type =', list_all_types ,'\n')

def all_config_neighbors(all_atom, radius):
    """ From all the atoms, return the list of the neighbors for each configuration and the maximum number of neighbors
    for each type of atom sorted alphabetically in a dictionary."""
    list_all_neighbors = []
    list_all_types = []
    for i in range(len(all_atom)) : 
        neighbors_atom_i = mol_get_neighbors(all_atom[i], radius)
        list_all_neighbors.append(neighbors_atom_i[0])
        list_all_types.append(neighbors_atom_i[1])
    list_all_types = max_type_neighbors(list_all_types)
    dic_all_types = dict()
    for key in sorted(list_all_types.items()):
        dic_all_types.update({key[0] : key[1]})
    return list_all_neighbors, dic_all_types

list_all_neighbors, dic_all_types = all_config_neighbors(atoms_2, 4)
print('Size of list_all_neighbors = ', len(list_all_neighbors))
print('Size of the elements of list_all_neighbors = ', len(list_all_neighbors[0]))
print('Size of the elements of the 1st element of list_all_neighbors = ', len(list_all_neighbors[0][0]))
print('List of all the neighbors type =', dic_all_types ,'\n')

Size of list_all_neighbors =  42
Size of the elements of list_all_neighbors =  27
List of all the neighbors type = {'O': 16, 'H': 5, 'Si': 8, 'C': 1, 'Al': 1} 

Size of list_all_neighbors =  2
Size of the elements of list_all_neighbors =  42
Size of the elements of the 1st element of list_all_neighbors =  26
List of all the neighbors type = {'Al': 1, 'C': 1, 'H': 5, 'O': 16, 'Si': 8} 



In [9]:
def class_neighbors(list_neighbors):
    """ From the list of the neighbors for a specific atom of reference, list_neighbors, return the distances from this 
    atom for each atom type in a dictionary."""
    list_neighbors = sorted(list_neighbors, key= itemgetter(1, 2))
    dic_distance_atom = {}
    for j in range(len(list_neighbors)):
        if list_neighbors[j][1] in dic_distance_atom : 
            dic_distance_atom[list_neighbors[j][1]].append(list_neighbors[j][2])
        else : 
            dic_distance_atom[list_neighbors[j][1]] = [list_neighbors[j][2]]
    return dic_distance_atom

print('List of the neighbors for a specific atom of reference:')
print(list_all_neighbors[1][41], '\n')
dic_atom_i = class_neighbors(list_all_neighbors[1][41])
print('Neighbors grouped by atom type and sorted in ascending order for the distance:')
print(dic_atom_i, '\n')

def get_list_distances(dic_atom_i,type_atom):
    """ From: - the dictionary of distances for each type of atom, from a atom of reference
              - the dictionary of the maximums of neighbors for each atom type, type_atom
    Return the list of the distances order by ascending. If there is no neighbor of a specific atom type, set 0."""
    distances = []
    for key, value in type_atom.items() :
        if key in dic_atom_i:
            for i in range(len(dic_atom_i[key])) : 
                distances.append(dic_atom_i[key][i])
            for i in range(len(dic_atom_i[key]), value) : 
                distances.append(0)
        else : 
            for i in range(value): 
                distances.append(0)
    return distances
print('List of the distance order by the previous dictionnary:')
print(get_list_distances(dic_atom_i,dic_all_types), '\n')


List of the neighbors for a specific atom of reference:
[[29, 'O', 1.5787762843840065], [30, 'O', 1.6792019993926566], [0, 'Al', 3.226658450990714], [21, 'O', 4.018323624679267], [27, 'O', 4.198413268295706], [25, 'O', 4.597726319507452], [19, 'O', 5.540916316997791], [14, 'O', 5.855860846368702], [38, 'Si', 6.077268148662403], [35, 'Si', 6.301345535078604], [24, 'O', 6.398725961359146], [22, 'O', 6.883358416437047], [7, 'O', 7.2637727153269545], [39, 'Si', 7.567778645771182], [12, 'O', 7.7819650009910974], [37, 'Si', 8.165082114215778], [18, 'O', 8.240781936512354], [26, 'O', 8.275467725041082], [23, 'O', 8.342630041336772], [17, 'O', 9.558257708994331], [31, 'Si', 10.370024419115662], [9, 'O', 10.615814078724782]] 

Neighbors grouped by atom type and sorted in ascending order for the distance:
{'Al': [3.226658450990714], 'O': [1.5787762843840065, 1.6792019993926566, 4.018323624679267, 4.198413268295706, 4.597726319507452, 5.540916316997791, 5.855860846368702, 6.398725961359146, 6.883

In [10]:
def create_row(list_all_neighbors,dic_all_types) : 
    """ From: - the list of all the neighbors of a configuration, list_all_neighbors
              - the dictionary of the maximums of neighbors for each atom type, type_atom
    Return a row vector with all the distances for all the atoms of references. """   
    distance_data = np.zeros((len(list_all_neighbors),sum(dic_all_types.values())))
    for i in range(len(list_all_neighbors)) : 
        dic_distance_atom_i = class_neighbors(list_all_neighbors[i])
        distance_data[i,:] = get_list_distances(dic_distance_atom_i,dic_all_types)
    #print(get_list_distances(dic_distance_atom_i,dic_all_types)) #Pour tester
    distance_data = np.ravel(distance_data)#, (1,len(list_all_neighbors[0])*sum(dic_all_types.values())))
    return distance_data


print('Creation of the row composed of all the distances of the neighbors for all the atom of reference (for one configuration):')
print(create_row(list_all_neighbors[1],dic_all_types)[-31:],'\n') #test for the following function

Creation of the row composed of all the distances of the neighbors for all the atom of reference (for one configuration):
[ 3.22665845  0.          0.          0.          0.          0.
  0.          1.57877628  1.679202    4.01832362  4.19841327  4.59772632
  5.54091632  5.85586085  6.39872596  6.88335842  7.26377272  7.781965
  8.24078194  8.27546773  8.34263004  9.55825771 10.61581408  6.07726815
  6.30134554  7.56777865  8.16508211 10.37002442  0.          0.
  0.        ] 



In [11]:
def data_array(all_atom, radius) : 
    """ From the atoms extraced from ASE in the feature all_atom and the radius, return an numpy array
    with all the distances for all the atoms of references. """ 
    array_neighbors, dic_all_types = all_config_neighbors(all_atom, radius)
    final_data = np.zeros((len(all_atom),42*sum(dic_all_types.values())))
    for i in range(len(array_neighbors)) : 
       final_data[i,:] = create_row(array_neighbors[i], dic_all_types)
    return final_data, dic_all_types

print('Final data array:')
print(data_array(atoms_2, 4),'\n')

print('Final data array of the last atom of reference (test for the previous function):')
print(data_array(atoms_2, 4)[0][1,31*41:])

Final data array:
(array([[0.        , 4.17265254, 2.38862119, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 4.17611891, 2.38794089, ..., 0.        , 0.        ,
        0.        ]]), {'Al': 1, 'C': 1, 'H': 5, 'O': 16, 'Si': 8}) 

Final data array of the last atom of reference (test for the previous function):
[ 3.22665845  0.          0.          0.          0.          0.
  0.          1.57877628  1.679202    4.01832362  4.19841327  4.59772632
  5.54091632  5.85586085  6.39872596  6.88335842  7.26377272  7.781965
  8.24078194  8.27546773  8.34263004  9.55825771 10.61581408  6.07726815
  6.30134554  7.56777865  8.16508211 10.37002442  0.          0.
  0.        ]


<a name="launch"></a>
## Dataset creation

First of all, we convert our data whose type is *atoms* into a *panda* data frame.
The 2nd file downloaded above is our target value. We will just make transformation of this value : for our future predictions, we will try to predict the *adsorption energy*, difference between this value and the potential energy of the molecule depending on his structure. We also subtract the mean of the adsorption energy in order to center the values.

In [12]:
def data_frame(all_atom, radius) : 
    """ From the atoms extraced from ASE in the feature all_atom and the radius, return the final data frame."""
    start = timeit.default_timer()
    
    # Features
    data, dic_all_types = data_array(all_atom, radius)
    name = []
    for key, val in dic_all_types.items() :
        for i in range(val) : 
            name.append(str(key)+' '+str(i+1)) 
    name = sorted(name, key=itemgetter(0))
    distance_table = pd.DataFrame(columns = name*42 )
    df2 = pd.DataFrame(data, columns =  name*42 )
    distance_table = distance_table.append(df2)
    
    # Target - adsorption energy
    adsorption_energy = []
    energy_data = pd.Series.tolist(data_energy.Energy)
    for i in range(len(atoms_200)): # Get energy of atoms
        adsorption_energy.append(atoms_200[i].get_potential_energy() - energy_data[i])
    mean_energy = np.mean(adsorption_energy)
    adsorption_energy = [adsorption_energy[i] - mean_energy for i in range(len(adsorption_energy))] # Subtract the mean of the adsorption energy   
    distance_table['Energy'] = pd.Series(adsorption_energy)
    
    stop = timeit.default_timer()
    print("Processing time: %0.2f s" % (stop - start))
    return distance_table

In [13]:
data_frame(atoms_2, 4)

Processing time: 6.99 s


,Al 1,C 1,H 1,H 2,H 3,H 4,H 5,O 1,O 2,O 3,...,O 16,Si 1,Si 2,Si 3,Si 4,Si 5,Si 6,Si 7,Si 8,Energy
0,0.0,4.172653,2.388621,3.504808,3.878122,0.0,0.0,1.687689,1.712646,1.741808,...,0.000000,6.072488,6.290598,7.550399,8.146431,10.333766,0.0,0.0,0.0,-0.006111
1,0.0,4.176119,2.387941,3.473500,3.831084,0.0,0.0,1.676656,1.688743,1.815746,...,10.615814,6.077268,6.301346,7.567779,8.165082,10.370024,0.0,0.0,0.0,-0.046628


<a name="launch"></a>
## Dataset copy

We will copy the final dataset. The goal is to store it and to save computation time: when we will use it, we shall not run the kernel.  
We save it in [pickle](https://docs.python.org/3/library/pickle.html#module-pickle) format.

In [14]:
# The below script lasts around 5-10 minutes
data_distance = data_frame(atoms_200, 4)
data_distance.head()
data_distance.to_pickle("./data_distance.pkl")

Processing time: 375.84 s


In [15]:
data_distance_new = pd.read_pickle("./data_distance.pkl")
data_distance_new.head()

,Al 1,C 1,H 1,H 2,H 3,H 4,H 5,O 1,O 2,O 3,...,Si 1,Si 2,Si 3,Si 4,Si 5,Si 6,Si 7,Si 8,Si 9,Energy
0,0.0,4.172653,2.388621,3.504808,3.878122,0.000000,0.0,1.687689,1.712646,1.741808,...,6.072488,6.290598,7.550399,8.146431,10.333766,0.000000,0.0,0.0,0.0,-0.006111
1,0.0,4.568290,2.468720,4.022415,4.214737,0.000000,0.0,1.703726,1.713953,1.716739,...,4.459793,6.039066,6.220273,7.574809,8.109587,10.501410,0.0,0.0,0.0,-0.046628
2,0.0,4.513681,2.613852,4.020059,4.122451,6.014900,0.0,1.628690,1.706415,1.746130,...,4.435719,5.857652,6.261540,7.578781,8.146058,10.294889,0.0,0.0,0.0,-0.070909
3,0.0,4.343103,2.359981,3.557888,4.507440,6.388042,0.0,1.690680,1.707743,1.758357,...,4.573852,6.070746,6.265656,7.507522,8.151083,10.237999,0.0,0.0,0.0,-0.036038
4,0.0,4.525015,2.542304,3.917335,4.521628,5.731700,0.0,1.670785,1.685060,1.719032,...,4.528103,5.822906,6.160028,7.251170,8.275507,10.157833,0.0,0.0,0.0,-0.030054


In [16]:
inv_data_distance = 1/data_distance_new
inv_data_distance = inv_data_distance.replace([np.inf, -np.inf], 0)
inv_data_distance.Energy = data_distance.Energy
inv_data_distance.to_pickle("./inv_data_distance.pkl")
inv_data_distance.head()

,Al 1,C 1,H 1,H 2,H 3,H 4,H 5,O 1,O 2,O 3,...,Si 1,Si 2,Si 3,Si 4,Si 5,Si 6,Si 7,Si 8,Si 9,Energy
0,0.0,0.239656,0.418652,0.285322,0.257857,0.000000,0.0,0.592526,0.583892,0.574116,...,0.164677,0.158967,0.132443,0.122753,0.096770,0.000000,0.0,0.0,0.0,-0.006111
1,0.0,0.218900,0.405068,0.248607,0.237263,0.000000,0.0,0.586949,0.583447,0.582500,...,0.224226,0.165589,0.160765,0.132017,0.123311,0.095225,0.0,0.0,0.0,-0.046628
2,0.0,0.221549,0.382577,0.248753,0.242574,0.166254,0.0,0.613991,0.586024,0.572695,...,0.225443,0.170717,0.159705,0.131947,0.122759,0.097136,0.0,0.0,0.0,-0.070909
3,0.0,0.230250,0.423732,0.281066,0.221855,0.156542,0.0,0.591478,0.585568,0.568713,...,0.218634,0.164724,0.159600,0.133200,0.122683,0.097675,0.0,0.0,0.0,-0.036038
4,0.0,0.220994,0.393344,0.255276,0.221159,0.174468,0.0,0.598521,0.593451,0.581723,...,0.220843,0.171736,0.162337,0.137909,0.120839,0.098446,0.0,0.0,0.0,-0.030054


---
Back to [top](#top).